In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=af01212e046e2f2dae5d98908cdfa7aa3cf47e45940a3d4fa7da16abf5d0a1a3
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=a9c2bc099ddde24d1a950a698757d95e1e270b3d3a66229ccdadbc583f859f56
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255
test_images = test_images/255

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
  keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
      kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
      activation='relu',
      input_shape=(28,28,1)
  ),
  keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter', min_value=32, max_value=64,  step=16),
      kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
      activation='relu'
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(
      units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
      activation='relu'
  ),
  keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output', project_name='Mnist Fashion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.8661666512489319

Best val_accuracy So Far: 0.9103333353996277
Total elapsed time: 00h 03m 54s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2211920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,262,282
Trainable params: 2,262,282
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1278 - accuracy: 0.9519 - val_loss: 0.2919 - val_accuracy: 0.9093
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0891 - accuracy: 0.9674 - val_loss: 0.2862 - val_accuracy: 0.9110
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0621 - accuracy: 0.9773 - val_loss: 0.3755 - val_accuracy: 0.9072
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0459 - accuracy: 0.9831 - val_loss: 0.3606 - val_accuracy: 0.9137
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0342 - accuracy: 0.9875 - val_loss: 0.4783 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0262 - accuracy: 0.9910 - val_loss: 0.5398 - val_accuracy: 0.9087
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0234 - accuracy: 0.9919 - val_loss: 0.5688 - val_accuracy